# Quantum Connection Setup

In [ ]:
from pathlib import Path
import sys, importlib

sys.path.append(str(Path.cwd().resolve() / "src"))

from scenarios import *

def reload_modules():
    global longRange, direct, comparison
    if 'plot_longRange' in sys.modules:
        longRange = importlib.reload(sys.modules['plot_longRange'])
    else:
        import plot_longRange as longRange

    if 'plot_directComm' in sys.modules:
        direct = importlib.reload(sys.modules['plot_directComm'])
    else:
        import plot_directComm as direct
    if 'plot_comparison' in sys.modules:
        comparison = importlib.reload(sys.modules['plot_comparison'])
    else:
        import plot_comparison as comparison

# Analysis Functions

In [2]:
import re
def get_img_path(label:str):
    # resolve output directory relative to this file (avoid changing cwd)
    base = Path.cwd().resolve()  # repo root
    subfolder = re.sub(r'\s+', '_', label.splitlines()[0])
    outdir = base / "img" / subfolder
    label = label.split(maxsplit=2)[0] + "_" + label.splitlines()[1]
    outdir.mkdir(parents=True, exist_ok=True)
    label = re.sub(r'\s+', '_', str(label)).strip()          # collapse whitespace/newlines
    label = re.sub(r'[^A-Za-z0-9._-]', '_', label)           # keep a safe subset of chars
    label = re.sub(r'[()]', '', label)
    label = label[:190] + ".png"
    imgpath = outdir / label
    return imgpath 

# Run Simulation

In [ ]:
reload_modules()
all_res_long = longRange.run_longrange_sims(param_sets)

Running long-range simulations...

=== (Long-range) Set 1/6: Zero length loss fibre (0.5 init, 0.0dB/km, T1=0.0us, T2=0.0us) (2000 shots) ===
  Distance 5 km, shots = 2000, p_ge = 0.50000
  Distance 20 km, shots = 2000, p_ge = 0.50000
  Distance 50 km, shots = 2000, p_ge = 0.50000

=== (Long-range) Set 2/6: High length loss fibre (0.0 init, 0.5dB/km, T1=0.0us, T2=0.0us) (1000 shots) ===
  Distance 5 km, shots = 1000, p_ge = 0.56234
  Distance 20 km, shots = 1000, p_ge = 0.10000
  Distance 50 km, shots = 1000, p_ge = 0.00316

=== (Long-range) Set 3/6: Low-Noise fibre (T1 = 500km travel time) (ideal memories) (0.0 init, 0.2dB/km, T1=2500.0us, T2=250.0us) (1000 shots) ===
  Distance 5 km, shots = 1000, p_ge = 0.79433
  Distance 20 km, shots = 1000, p_ge = 0.39811
  Distance 50 km, shots = 1000, p_ge = 0.10000

=== (Long-range) Set 4/6: Low-Noise fibre (T1 = 500km travel time) (noisy memories) (0.0 init, 0.2dB/km, T1=2500.0us, T2=250.0us) (1000 shots) ===
  Distance 5 km, shots = 1000, p_g

In [ ]:
import copy
param_direct = copy.deepcopy(param_sets)
for p in param_direct:
    p["distances"] = [2*d for d in p["distances"]]
autofill_params(param_direct)

reload_modules()
all_res_direct = direct.run_sims(param_direct, 1e-3)

Zero length loss fibre (10km) p_ge = 0.5
Zero length loss fibre (40km) p_ge = 0.5
Zero length loss fibre (100km) p_ge = 0.5
High length loss fibre (10km) p_ge = 0.31622776601683794
High length loss fibre (40km) p_ge = 0.01
High length loss fibre (100km) p_ge = 1e-05
Low-Noise fibre (T1 = 500km travel time) (ideal memories) (10km) p_ge = 0.6309573444801932
Low-Noise fibre (T1 = 500km travel time) (ideal memories) (40km) p_ge = 0.15848931924611134
Low-Noise fibre (T1 = 500km travel time) (ideal memories) (100km) p_ge = 0.01
Low-Noise fibre (T1 = 500km travel time) (noisy memories) (10km) p_ge = 0.6309573444801932
Low-Noise fibre (T1 = 500km travel time) (noisy memories) (40km) p_ge = 0.15848931924611134
Low-Noise fibre (T1 = 500km travel time) (noisy memories) (100km) p_ge = 0.01
High-Noise fibre (T1 = 50km travel time) (10km) p_ge = 0.6309573444801932
High-Noise fibre (T1 = 50km travel time) (40km) p_ge = 0.15848931924611134
High-Noise fibre (T1 = 50km travel time) (100km) p_ge = 0.01
E

In [34]:
reload_modules()
direct.plot_sims(all_res_direct)
longRange.plot_longrange(all_res_long)

#####Zero length loss fibre (0.5 init, 0.0dB/km, T1=0.0us, T2=0.0us) (2000 shots)#####
#####High length loss fibre (0.0 init, 0.5dB/km, T1=0.0us, T2=0.0us) (1000 shots)#####
#####Low-Noise fibre (T1 = 500km travel time) (ideal memories) (0.0 init, 0.2dB/km, T1=2500.0us, T2=250.0us) (1000 shots)#####
#####Low-Noise fibre (T1 = 500km travel time) (noisy memories) (0.0 init, 0.2dB/km, T1=2500.0us, T2=250.0us) (1000 shots)#####
#####High-Noise fibre (T1 = 50km travel time) (0.0 init, 0.2dB/km, T1=250.0us, T2=25.0us) (1000 shots)#####
#####Extreme-Noise fibre (T1 = 5km travel time) (0.0 init, 0.2dB/km, T1=25.0us, T2=2.5us) (1000 shots)#####


In [ ]:
reload_modules()
comparison.print_comparison_table(all_res_long, all_res_direct)
# comparison.plot_comparison(all_res_long, all_res_direct)


COMPARISON TABLE: Repeater vs Direct Communication
Scenario                       Distance     Method               Avg Time Units     Avg Fidelity    P_gen     
------------------------------------------------------------------------------------------------------------------------
Extreme-Noise fibre              10km       Repeater             1.44               0.6064          0.7943    
                                            Direct               1.61               0.5328          0.6310    
------------------------------------------------------------------------------------------------------------------------
Extreme-Noise fibre              40km       Repeater             3.37               0.7030          0.3981    
                                            Direct               6.29               0.5001          0.1585    
------------------------------------------------------------------------------------------------------------------------
Extreme-Noise fibre           